In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from white_demura import mean_pooling, run_dfs, run_dfs_green, reconstruct_mask, RGB_mask_no_roi, RGB_mask, sum_contour, read_and_extract_roi, apply_mask, get_signal_mapping, generate_mapping, extract_middle_roi


In [2]:
from read_images import generate_RGB_dataframes

In [3]:
output_path_root = '/Users/codyyu/Desktop/0109/output/'

In [4]:
thresholds = [5, 7, 10, 12, 15]

In [5]:
red_image_path = '/Users/codyyu/Desktop/monoCapture/3_8/rCapMap3.8.tif'
green_image_path = '/Users/codyyu/Desktop/monoCapture/3_8/gCapMap3.8.tif'
blue_image_path = '/Users/codyyu/Desktop/monoCapture/3_8/bCapMap3.8.tif'
clear_image_path = '/Users/codyyu/Desktop/monoCapture/3_8/wCapMap3.8.tif'



In [7]:
def run(th, output_path):
    red_image_255 = cv2.imread(red_image_path, cv2.IMREAD_GRAYSCALE) 
    green_image_255 = cv2.imread(green_image_path, cv2.IMREAD_GRAYSCALE) 
    blue_image_255 = cv2.imread(blue_image_path, cv2.IMREAD_GRAYSCALE) 
    clear_image_255 = cv2.imread(clear_image_path, cv2.IMREAD_GRAYSCALE) 

    red_binary = np.where(red_image_255 > th, 1, 0)
    green_binary = np.where(green_image_255 > th, 1, 0)
    blue_binary = np.where(blue_image_255 > th, 1, 0)


    rg_ol = red_binary * green_binary
    rb_ol = red_binary * blue_binary
    gb_ol = green_binary * blue_binary
    rgb_ol = red_binary * green_binary * blue_binary

    red_image_255 = red_image_255.astype('float64')
    green_image_255 = green_image_255.astype('float64')
    blue_image_255 = blue_image_255.astype('float64')



    red_binary = red_binary.astype('float64')
    green_binary = green_binary.astype('float64')
    blue_binary = blue_binary.astype('float64')



    for i in (range(len(red_binary))):
        for j in (range(len(red_binary[0]))):
            if rg_ol[i][j] == 1 or rb_ol[i][j] == 1:
                sum = 0.0
                sum = red_image_255[i][j] + green_image_255[i][j] + blue_image_255[i][j]
                if sum != 0:
                    red_binary[i][j] = red_image_255[i][j] / sum
                #print(red_image_255[i][j], sum, red_image_255[i][j] / sum)


    for i in (range(len(green_binary))):
        for j in (range(len(green_binary[0]))):
            if rg_ol[i][j] == 1 or gb_ol[i][j] == 1:
                sum = 0.0
                sum = red_image_255[i][j] + green_image_255[i][j] + blue_image_255[i][j]
                if sum != 0:
                    green_binary[i][j] = green_image_255[i][j] / sum



    for i in (range(len(blue_binary))):
        for j in (range(len(blue_binary[0]))):
            if gb_ol[i][j] == 1 or rb_ol[i][j] == 1:
                sum = 0.0
                sum = red_image_255[i][j] + green_image_255[i][j] + blue_image_255[i][j]
                if sum != 0:
                    blue_binary[i][j] = blue_image_255[i][j] / sum



    red_filtered = np.where(red_image_255 > th, 1, 0) * red_image_255
    green_filtered = np.where(green_image_255 > th, 1, 0) * green_image_255
    blue_filtered = np.where(blue_image_255 > th, 1, 0) * blue_image_255
    



    cv2.imwrite(output_path + 'red_filtered.png', red_filtered)
    cv2.imwrite(output_path + 'green_filtered.png', green_filtered)
    cv2.imwrite(output_path + 'blue_filtered.png', blue_filtered)



    red_mask_filtered_ol = (red_binary * clear_image_255.astype('float64')).astype('uint8')
    green_mask_filtered_ol = (green_binary * clear_image_255.astype('float64')).astype('uint8')
    blue_mask_filtered_ol = (blue_binary * clear_image_255.astype('float64')).astype('uint8')


    cv2.imwrite(output_path + 'red_mask_filtered.png', red_mask_filtered_ol)
    cv2.imwrite(output_path + 'green_mask_filtered.png', green_mask_filtered_ol)
    cv2.imwrite(output_path + 'blue_mask_filtered.png', blue_mask_filtered_ol)



    red_float_mask = (red_binary * 255.0).astype('uint8')
    green_float_mask = (green_binary * 255.0).astype('uint8')
    blue_float_mask = (blue_binary * 255.0).astype('uint8')
    cv2.imwrite(output_path + 'red_float_mask.png', red_float_mask)
    cv2.imwrite(output_path + 'green_float_mask.png', green_float_mask)
    cv2.imwrite(output_path + 'blue_float_mask.png', blue_float_mask)


In [8]:
for th in thresholds:
    run(th, output_path_root + str(th) + '/')